## Gig Economy Project

By: Janet Taylor & Evan Kessler

Workers in the gig economy (i.e., Uber, GrubHub, TaskRabbit, etc.) depend on the reviews they get to continue working on the platform. Workers on those platforms are often affected by receiving negative reviews that describe elements that were not under their control. The project explores how the analysis of online reviews can detect when unfair reviews are left to workers and identify the type of unfair review that was left.

In [1]:
import pandas as pd

In [7]:
uber = pd.read_csv("uber_cleaned.csv")
lyft = pd.read_csv("lyft_cleaned.csv")
pd.options.display.max_colwidth = 500
uber["Review Body"].head()

0    all your driver's we very good, knowledgeable and friendly. however, ellie was exceptional. she established a rapport with my daughter and i in the very short time we were in the car with her. she was helpful, answered our questions about the island and even gave us a few suggestions on places she enjoyed eating. which in fact we did try freda's and it was good. thank you, ellie, for your service, customer service and helpfulness!
1                                                                                                                                                                driver canceled the ride because he couldn’t find us and we were charged $5.  we were waiting for him but he didn’t show up.  when i tried contacting him, i went directly into voicemail but couldn’t leave a message.  had to order another ride and the second driver picked us up.
2                                                                                                                       